In [35]:
# Librerías necesarias
import pandas as pd
import re
import requests

In [3]:
df = pd.read_csv("ficheros/areas_ac.csv")
df.head()

,WKT,NOMBRE
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...
2,POINT (0.35921 42.90715 0.0),Arreau Ã¡rea AC municipal (2 eurosAgua:sivaci...
3,POINT (-0.41007 38.39479 0.0),Camper Area Campello Beach (14eur/nocheAgua:...
4,POINT (-8.44443 43.37179 0.0),A CoruÃ±a Gratuito: Si Pernocta: Si WC: Si Ag...


In [4]:

df.isnull().sum()

WKT       0
NOMBRE    0
dtype: int64

In [6]:
df.dtypes

WKT       object
NOMBRE    object
dtype: object

### EXTRAER LA INFORMACIÓN LAS COLUMNAS
---

In [12]:
# Función para sacar las coordenadas de la columna WKT
def sacar_latitud(col):
    patron = "-?\d+\.\d+" 
    return re.findall(patron, col)[1]

def sacar_longitud(col):
    patron = "-?\d+\.\d+" 
    return re.findall(patron, col)[0]

In [13]:
df["latitud"]= df["WKT"].apply(sacar_latitud)

In [14]:
df["longitud"] = df["WKT"].apply(sacar_longitud)

In [15]:
df.head(2)

,WKT,NOMBRE,latitud,longitud
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696


In [16]:
# Función para buscar si es de pago o no
def precio(col):
    
    patron1 = "[G|g]ratuito:*\s*[N|n]o"
    patron2 = "[G|g]ratuito"
    patron3 = ".*\d+"
    if re.findall(patron1,col):             
        return "De pago"
    elif re.findall(patron2,col):             
        return "Gratuito"
    elif re.findall(patron3,col):             
        return "De pago"
    else:
        return "Gratuito"

In [17]:
df["precio"] = df["NOMBRE"].apply(precio)

In [18]:
df.head(2)

,WKT,NOMBRE,latitud,longitud,precio
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito


In [19]:
df.precio.value_counts()

Gratuito    552
De pago     361
Name: precio, dtype: int64

In [20]:
# Función para saber si disponen de servicio de aguas o no
def buscar_agua(col):
    patron = "[A|a]gua:*\s*[N|n]o*"
    patron2 = "[A|a]gua:*\s*[S|s]*i*"
    if re.findall(patron, col):
        return "no"
    elif re.findall(patron2, col):
        return "si"
    else:
        return "desconocido"

In [21]:
df["agua"] = df["NOMBRE"].apply(buscar_agua)

In [22]:
df.head(2)

,WKT,NOMBRE,latitud,longitud,precio,agua
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito,si
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito,si


In [23]:
df["agua"].value_counts()

si             885
no              19
desconocido      9
Name: agua, dtype: int64

In [24]:
# Función para saber si tienen wc
def buscar_wc(col):
    patron = "[W|w][C|c]:*\s*[N|n]o*"
    patron2 = "[W|w][C|c]:*\s*[S|s]*i*"
    if re.findall(patron, col):
        return "no"
    elif re.findall(patron2, col):
        return "si"
    else:
        return "desconocido"

In [25]:
df["wc"] = df["NOMBRE"].apply(buscar_wc)

In [26]:
df.head(2)

,WKT,NOMBRE,latitud,longitud,precio,agua,wc
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito,si,si
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito,si,si


In [27]:
df.wc.value_counts()

no             511
si             363
desconocido     39
Name: wc, dtype: int64

In [28]:
# Función para saber si le han etiquetado el servicio de vaciado
def vaciado(col):
    patron = "[V|v]aciado:*\s*[N|n]o*"
    patron2 = "[V|v]aciado:*\s*[S|s]*i*"
    if re.findall(patron, col):
        return "no"
    elif re.findall(patron2, col):
        return "si"
    else:
        return "desconocido"

In [29]:
df["vaciado"] = df["NOMBRE"].apply(vaciado)

In [30]:
df.head(2)

,WKT,NOMBRE,latitud,longitud,precio,agua,wc,vaciado
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito,si,si,si
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito,si,si,desconocido


In [31]:
df.vaciado.value_counts()

si             629
desconocido    278
no               6
Name: vaciado, dtype: int64

In [40]:
# Después de extraer la información, elimino la columna WKT
df.drop(columns=["WKT", "NOMBRE"], axis=1, inplace= True)

KeyError: "['WKT'] not found in axis"

### API PARA EXTRAER LA LOCALIDAD, PROVINCIA Y PAIS
--- 

In [36]:
def localidad(lat, lon):
    
    # hacemos la llamada  a la API
    url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
   
    respuesta = response.json()
    direccion = respuesta["address"]
    lista = direccion.keys()
    
    # Dependiendo del país y del tamaño, vienen con diferente denominación
    if "town" in lista:        
        return direccion["town"]
    elif "city" in lista:
        return direccion["city"]
    elif "village" in lista:
        return direccion["village"]
    elif "municipality" in lista:
        return direccion["municipality"]
    else:
        return "desconocido"

In [37]:
df["población"] = df.apply(lambda pepe: localidad(pepe["latitud"], pepe["longitud"]), axis= 1)

In [38]:
df.head()

,NOMBRE,latitud,longitud,precio,agua,wc,vaciado,población
0,Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito,si,si,si,Carcaixent
1,Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito,si,si,desconocido,Boiro
2,Arreau Ã¡rea AC municipal (2 eurosAgua:sivaci...,42.90715,0.35921,De pago,si,no,si,Arreau
3,Camper Area Campello Beach (14eur/nocheAgua:...,38.39479,-0.41007,De pago,si,si,desconocido,el Campello
4,A CoruÃ±a Gratuito: Si Pernocta: Si WC: Si Ag...,43.37179,-8.44443,Gratuito,si,si,desconocido,A Coruña


In [39]:
df["población"].value_counts()

Encamp                  3
Thann                   2
Campan                  2
Roma                    2
Ljubljana               2
                       ..
Jumièges                1
Kalmar                  1
Karlskrona              1
Kaysersberg-Vignoble    1
Cúllar                  1
Name: población, Length: 887, dtype: int64

In [41]:
def region(lat, lon):
    
    # hacemos la llamada  a la API
    url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
   
    respuesta = response.json()
    direccion = respuesta["address"]
    lista = direccion.keys()
    
    if "state" in lista:
        return direccion["state"]
    elif "county" in lista:
        return direccion["county"]
    else:
        return direccion["country"]

In [42]:
df["region"] = df.apply(lambda pepe: region(pepe["latitud"], pepe["longitud"]), axis= 1)

In [43]:
df.head(2)

,NOMBRE,latitud,longitud,precio,agua,wc,vaciado,población,region
0,Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito,si,si,si,Carcaixent,Comunitat Valenciana
1,Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito,si,si,desconocido,Boiro,Galicia


In [44]:
df["region"].value_counts()

Nouvelle-Aquitaine     100
Occitanie               66
Catalunya               64
Galicia                 58
Andalucía               38
                      ... 
Puglia                   1
Norge                    1
England                  1
Comunidad de Madrid      1
San Marino               1
Name: region, Length: 111, dtype: int64

In [45]:
def pais(lat, lon):
    
    # hacemos la llamada  a la API
    url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}'

    response = requests.get(url=url)
    codigo_estado = response.status_code
    razon_estado = response.reason
   
    respuesta = response.json()
    direccion = respuesta["address"]
    lista = direccion.keys()
    
    return direccion["country"]

In [46]:
df["pais"] = df.apply(lambda pepe: pais(pepe["latitud"], pepe["longitud"]), axis= 1)

In [47]:
df.head(2)

,NOMBRE,latitud,longitud,precio,agua,wc,vaciado,población,region,pais
0,Area AC Hort de Soriano (Carcaixent) (Gratuit...,39.06956,-0.4098,Gratuito,si,si,si,Carcaixent,Comunitat Valenciana,España
1,Area de ac playa de BarraÃ±a (Boiro) (Gratuit...,42.64159,-8.89696,Gratuito,si,si,desconocido,Boiro,Galicia,España


In [48]:
df["pais"].value_counts()

España                            347
France                            343
Deutschland                        62
Italia                             59
Portugal                           37
Slovenija                          12
Norge                              10
Sverige                             8
Andorra                             6
Nederland                           5
Danmark                             5
België / Belgique / Belgien         5
Schweiz/Suisse/Svizzera/Svizra      2
Österreich                          2
United Kingdom                      2
Hrvatska                            2
Australia                           1
Suomi / Finland                     1
Lëtzebuerg                          1
Polska                              1
Slovensko                           1
San Marino                          1
Name: pais, dtype: int64

In [53]:
df.head(2)

,latitud,longitud,precio,agua,wc,vaciado,población,region,pais
0,39.06956,-0.4098,Gratuito,si,si,si,Carcaixent,Comunitat Valenciana,España
1,42.64159,-8.89696,Gratuito,si,si,desconocido,Boiro,Galicia,España


In [54]:
df = df.reindex(columns=["latitud", "longitud", "población", "region", "pais", "precio", "agua", "wc", "vaciado"])
df.head(2)

,latitud,longitud,población,region,pais,precio,agua,wc,vaciado
0,39.06956,-0.4098,Carcaixent,Comunitat Valenciana,España,Gratuito,si,si,si
1,42.64159,-8.89696,Boiro,Galicia,España,Gratuito,si,si,desconocido


In [55]:
df.to_csv("ficheros/areas_limpio.csv")